In [ ]:
import pandas as pd
from os import path
import csv
import numpy as np
from time import time
import numpy as np
import scipy.stats as stats

In [ ]:
## Load the test data 
t2d_22_file = 'C:/Users/saketh/stuff/Medlytics/Week3Public/BWSI_100set/T2D/Affx_gt_T2D_Chiamo_22.tped.gz'
ctrl1_22_file = 'C:/Users/saketh/stuff/Medlytics/Week3Public/BWSI_100set/58C/Affx_gt_58C_Chiamo_22.tped.gz' # 58C
ctrl2_22_file = 'C:/Users/saketh/stuff/Medlytics/Week3Public/BWSI_100set/NBS/Affx_gt_NBS_Chiamo_22.tped.gz' # NBS
## The 22 means Chromosome 22

t2d = pd.read_csv(t2d_22_file, compression='gzip', header=None, sep='\t')
ctrl1 = pd.read_csv(ctrl1_22_file, compression='gzip', header=None, sep='\t')
ctrl2 = pd.read_csv(ctrl2_22_file, compression='gzip', header=None, sep='\t')

#SNP code rsID and snpID files for chromosome 22
snp_file = 'C:/Users/saketh/stuff/Medlytics/Week3Public/BWSI_100set/58C/snps/snps_22'
snp = pd.read_csv(snp_file, header=None, sep='\t')

In [ ]:
t2d_header = t2d.loc[:, 0:3]
t2d_data = t2d.loc[:, 4:]
ctrl1_header = ctrl1.loc[:, 0:3]
ctrl1_data = ctrl1.loc[:, 4:]
ctrl2_header = ctrl2.loc[:, 0:3]
ctrl2_data = ctrl2.loc[:, 4:]
snp

In [ ]:
snpinfo = snp.loc[:, 3:4]
ctrl1_header.rename(columns={1:'SNP'}, inplace=True)
snpinfo.rename(inplace=True, columns={3:'SNP', 4:'rsID'})
snpmap = ctrl1_header.join(snpinfo.set_index('SNP'), on='SNP', how='left')
snpmap

In [220]:
snp1_ctrl1 = ctrl1_data.loc[1]
snp1_ctrl2 = ctrl2_data.loc[1]
snp1_t2d = t2d_data.loc[1]
snp1_t2d

4       C T
5       T T
6       C C
7       C C
8       C C
9       C T
10      C C
11      C C
12      C C
13      C C
14      C T
15      C C
16      C C
17      C T
18      C C
19      C T
20      C T
21      T T
22      C T
23      C T
24      C T
25      C T
26      T T
27      C T
28      C C
29      T T
30      C T
31      C T
32      C C
33      C T
       ... 
1973    C T
1974    C C
1975    C T
1976    C T
1977    C T
1978    C T
1979    C T
1980    C T
1981    T T
1982    T T
1983    C T
1984    C T
1985    C C
1986    C T
1987    C T
1988    T T
1989    C T
1990    C T
1991    T T
1992    C C
1993    C C
1994    C T
1995    C C
1996    C T
1997    C C
1998    T T
1999    C T
2000    C T
2001    C T
2002    C T
Name: 1, Length: 1999, dtype: object

In [212]:
def allele_frq(snp):
    '''
    Pass a single SNP and get in return
    a list of all of the number of alleles in the order
    A, C, G, T
    Returns minor allele frequency, minor allele, and major allele respectively.
    '''
    snp_series = list(snp.T)
    total_alleles = 0
    c = 0
    g = 0
    a = 0
    t = 0

    # My Shit Code
    for i in snp_series:
        total_alleles += 2
        for j in i:
            if j == 'C':
                c += 1
            elif j == 'A':
                a += 1
            elif j == 'G':
                g += 1
            elif j == 'T':
                t += 1
            else:
                pass
    freq_list = [i for i in [float(a/total_alleles), float(c/total_alleles), float(g/total_alleles), float(t/total_alleles)]]
    freqno0 = [i for i in [float(a/total_alleles), float(c/total_alleles), float(g/total_alleles), float(t/total_alleles)] if i != 0.0]
    minor_freq = np.min(freqno0)
    minor_freq_index = freq_list.index(minor_freq)
    major_freq_index = np.argmax(freq_list)
    retval = []
    retval.append(minor_freq)
    if minor_freq_index == 0:
        retval.append('A')
    elif minor_freq_index == 1:
        retval.append('C')
    elif minor_freq_index == 2:
        retval.append('G')
    else:
        retval.append('T')
    if major_freq_index == 0:
        retval.append('A')
    elif major_freq_index == 1:
        retval.append('C')
    elif major_freq_index == 2:
        retval.append('G')
    else:
        retval.append('T')
    retval.append(total_alleles)
    return retval

def conf_matrix(snp_healthy, snp_disease):
    '''
    Pass a healthy SNP and a disease SNP and get in return
    a list of all of the numbers of alleles in the order
    A, C, G, T
    '''
    snp_series_healthy = list(snp_healthy.T)
    snp_series_disease = list(snp_disease.T)
    total_alleles_h = 0
    total_alleles_d = 0
    c_h = 0
    g_h = 0
    a_h = 0
    t_h = 0
    c_d = 0
    g_d = 0
    a_d = 0
    t_d = 0
    
    # My Shit Code
    for i in snp_series_healthy:
        total_alleles_h += 2
        for j in i:
            if j == 'C':
                c_h += 1
            elif j == 'A':
                a_h += 1
            elif j == 'G':
                g_h += 1
            elif j == 'T':
                t_h += 1
            else:
                pass
    
    # My Shit Code
    for i in snp_series_disease:
        total_alleles_d += 2
        for j in i:
            if j == 'C':
                c_d += 1
            elif j == 'A':
                a_d += 1
            elif j == 'G':
                g_d += 1
            elif j == 'T':
                t_d += 1
            else:
                pass
    retval = np.array([i for i in [a_d, c_d, g_d, t_d, a_h, c_h, g_h, t_h] if i != 0]).reshape((2, 2))
    return retval

In [213]:
def odds_ratio(snp_healthy, snp_disease):
    disease_count = allele_frq(snp_disease)[0]
    healthy_count = allele_frq(snp_healthy)[0]
    ratio = (disease_count*(1-healthy_count))/(healthy_count*(1-disease_count))
    return(ratio)

In [221]:
odds_ratio(snp1_ctrl1, snp1_t2d)

1.1019003978402955

In [222]:
stats.chi2_contingency(conf_matrix(snp1_ctrl1, snp1_t2d))[1]
#p - value of OR -> 0.96

0.049078217495074614

In [217]:
# HWE equilibrium p-value
def HWE(snp):
    snp_string = snp.to_string()
    #COUNT C C
    #C T
    #T T

    a = snp_string.count('C C')
    b = snp_string.count('C T')
    c = snp_string.count('T T')
    t = snp_string.count('\n') # total number of ppl


    obs = np.array([a,b,c])
    exp = np.array([allele_frq(snp1_ctrl1)[0] ** 2, 2 * allele_frq(snp1_ctrl1)[0] * (1-allele_frq(snp1_ctrl1)[0]), (1-allele_frq(snp1_ctrl1)[0]) ** 2]) * t
    HWE = ((obs - exp) ** 2)/exp
    return stats.distributions.chi2.sf(np.sum(HWE), 1)

In [218]:
HWE(snp1_ctrl1)

0.9124994962314105